<a href="https://colab.research.google.com/github/edcalderin/DeepLearning_SaturdaysAI/blob/master/Tareas/Tarea_Transformers_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea: Advanced Topics
### Grupo: XXXXX
Integrantes:
<br>
* Integrante 1 
* Integrante 2
* Integrante 3
* Integrante 4
<br>

Indicaciones:
<rb>
* Debe realizar la siguiente tarea hasta el domingo 27 de junio, 23:59 UTC - 4
* Debe hacer una copia de este notebook para poder editar el código.
* Una vez finalizado el trabajo debe subir el link de su notebook (con permisos de lector) en la sección de "Tareas" del Módulo 5: Advanced Topics.

# Transformers para análisis de sentimiento

En la clase práctica de Transformers se definió una clase para tratamiento de problemas Seq2Seq y también se analizó como usar los transformers para la clasificación. En el ejercicio de esta tarea utilizarán la clase Seq2SeqTransformer para la clasificación.

Para ellos utilizaremos los datos de IMDB, de comentarios de películas en inglés y los clasificaremos en positivos o negativos.

Para más detalles de este problema y otras formas de resolverlos vea: [análisis de sentimiento con pytorch en github](https://github.com/bentrevett/pytorch-sentiment-analysis)

## Ejercicio: 

Crea una clase Seq2ClassTransformer a partir de  Seq2SeqTransformer que pueda ser utilizada para la clasificación. Utiliza una de las dos variantes mencionadas en clase:

1.   Adiciona el clasificador al final del codificador mediante el uso de una capa MLP.
2.   Adiciona el token de clasificación como parte de la secuencia, como el ejemplo visto en clase de la clasificación de imágenes.

Haz los cambios necesarios para que el código funcione. Entrena el modelo utilizando varios valores para los hiperparámetros:

* EMB_SIZE = {256, 512}
* NHEAD = {4, 8}
* NUM_ENCODER_LAYERS = {2, 3}


Presenta los gráficos en tiempo de entrenamiento, de la precisión y pérdida (accuracy y loss) en el conjunto de validación y de entrenamiento, para cada una de las épocas.
Presenta los datos de testing en una tabla para cada una de las combinaciones


La entrega de la tarea será el notebook con el código y los gráficos. Por favor utilicen la misma semilla (SEED) para ejecutar su código para garantizar reproducibilidad.

In [ ]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
! pip install transformers

In [ ]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

In [ ]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

In [ ]:
from torchtext.legacy import data

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [ ]:
LABEL.build_vocab(train_data)

In [ ]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## Construcción del modelo

In [ ]:
import math
import torchtext
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
from torch import Tensor
import io
import time

In [ ]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
        
    def forward(self, tokens: Tensor):
        embedding = self.embedding(tokens.long())
        return embedding * math.sqrt(self.emb_size)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        pos_embedding = self.pos_embedding[:token_embedding.size(0),:]
        embedding = token_embedding + pos_embedding
        return self.dropout(embedding)

In [ ]:
def create_mask(src):
  src_seq_len = src.shape[0]
  src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

  src_padding_mask = (src == pad_token_idx).transpose(0, 1)
  return src_mask, src_padding_mask

In [ ]:
# Crea la clase Seq2ClassTransformer Aquí, tomando como ejemplo la clase Seq2SeqTransformer vista en clase

In [ ]:
SRC_VOCAB_SIZE = len(tokenizer.vocab)
EMB_SIZE = 256
NHEAD = 4
FFN_HID_DIM = 256
NUM_ENCODER_LAYERS = 3

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

transformer = # Completa el código del modelo

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(
    transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9
)

loss = loss.to(device)

In [ ]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        
        # Crea la máscara
        predictions = # Completa la llamada al modelo
        

        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            # Crea la máscara
            predictions = # Completa la llamada al modelo
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(transformer, train_iterator, optimizer, loss)
    valid_loss, valid_acc = evaluate(transformer, valid_iterator, loss)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
model.load_state_dict(torch.load('transformer.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

## Inferencia

In [ ]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
predict_sentiment(transformer, tokenizer, "This film is terrible")

In [ ]:
predict_sentiment(transformer, tokenizer, "This film is great")